# Module 2: Linux for Robotics
## In-Class Exercise 2 - Linux
---
### A note on this document
This document is known as a Jupyter notebook; it is used in academia and industry to allow text and executable code to coexist in a very easy to read format. Blocks can contain text or code, and for blocks containing code, press `Shift + Enter` to run the code. Earlier blocks of code need to be run for the later blocks of code to work.

### Purpose
This Jupyter Notebook accompanies the introduction to Linux notetaker used in class. We will apply the knowledge you learned by interacting with the Ubuntu Operating System (OS) on the Master.

### Linux Commands
During class we went over a number of basic Linux commands. Open a terminal on the Master and let's practice using those commands (use the shortcut `ctrl+alt+t` to open a new terminal window or select an open terminal and hit `ctrl+shift+t` to open a new terminal tab).

When observing the terminal (or Shell) you will note the syntax `username@hostname:`(i.e., on the master: `dfec@masterX:`, on the robot: `pi@robotX`), the current working directory (i.e., `~`, which represents the home folder of the user), and lastly the '$' character and a blinking cursor. This line is the prompt and the blinking cursor indicates the terminal is active and ready for commands.

Run the command to make a new directory:

`mkdir my_folder`

**Pro tip:** In Linux, if you highlight a command you can paste it into the terminal by clicking the scroll wheel.

Change directories into your new folder:

`cd my_folder`

Create a new script we can use to drive the robot:

`touch move_usafabot.sh`

We can use the Nano text editor to edit a file within the terminal. There are a number of text editors within the terminal and WWIII might be fought over which is best; some other options include Vim and Emacs. Nano is one of the more simple editors for quick edits. Feel free to use whichever works best for you, but all guidance within this course will be based on Nano.

A bash script is a regular text file that allows you to run any command you would run within the terminal. We will use it to run a few ROS command line tools.

Edit the new bash script:

`nano move_usafabot.sh`

Copy the following into the script:

```bash
#!/bin/bash

ARG1=$1

if [ $ARG1 == 'forward' ]; then
    rostopic pub /cmd_vel geometry_msgs/Twist "linear:
    x: 0.15
    y: 0.0
    z: 0.0
angular:
    x: 0.0
    y: 0.0
    z: 0.0"
        
elif [ $ARG1 == 'rotate' ]; then
    rostopic pub /cmd_vel geometry_msgs/Twist "linear:
    x: 0.0
    y: 0.0
    z: 0.0
angular:
    x: 0.0
    y: 0.0
    z: 0.5"
        
elif [ $ARG1 == 'stop' ]; then
    rostopic pub /cmd_vel geometry_msgs/Twist "linear:
    x: 0.0
    y: 0.0
    z: 0.0
angular:
    x: 0.0
    y: 0.0
    z: 0.0"
else
echo "Please enter one of the following:
    forward
    rotate
    stop"
fi
```

Typing `ctrl+s` saves the file and then typing `ctrl+x` exits Nano.

Before running this script, let's get our ROS environment setup:

1. Open a new terminal and type `roscore`.
2. Open a new terminal tab and run our USAFABot simulation: `roslaunch usafabot_gazebo usafabot_gazebo.launch world:=empty.world`

Now run the script:

`./move_usafabot.sh`

Did you get an error? That is because the permissions have not been properly set and you do not have execute permissions. You can observe the permissions of a file by typing `ls -la`.

For the `move_usafabot.sh` file you should see `-rw-rw-r--`. The first position indicates file type ('d' for directory, '-' for regular file), the next three positions indicate read (r), write (w), and execute (x) permissions for the file owner (u), the next three indicate permisions for the group owner of the file (g), and the last three characters indicate permissions for all other users (o). Permissions can be controlle for all users (a) as well.

You can change the permissions of a file or directory using the command `chmod`:

>**Syntax:** `chmod <groups to assign the permissions><permissions to assign/remove> <file/folder>`

For example, if we wanted to give the Owner execution permissions, you can enter the command:

`chmod u+x move_usafabot.sh` 

Typically we will give all users executable permissions (`chmod +x move_usafabot.sh`). This isn't the most secure thing to do, but in our controlled environment, it isn't an issue. If you type `ls -la` now you should see the 'x' permission added for each permission group.

Try running your script again:

`./move_usafabot.sh rotate`

You can remove permissions by utilizing the '-' character instead of '+'.

Now create a new bash script, `bash_script.sh`, to accomplish the following:
1. Moves to the folder `~/master_ws/src/usafabot/usafabot_curriculum/`
1. Creates a directory called `my_scripts`
1. Moves into that directory
1. Create a file called **move_usafabot_square.py**
1. Lists all files showing the permissions
1. Modify the permissions of the **move_usafabot_square.py** file so all groups have executable permissions
1. List all files again showing the updated permissions

We are done with this script so let's remove it. The `rm` command can remove folders or files. If you want to learn more about a command there are tool tools: Help: `rm --help`; Manual: `man rm`.

Type the following to remove our bash script: `rm bash_script.sh`. To delete a whole folder add the `-r` tag to remove directories and thier contents recursively (e.g., `rm -r my_folder`, but don't remove your folder just yet).

We can copy (`cp`, just like ctrl+c in a GUI) and move (`mv`, just like ctrl+x in a GUI) files and folders as well. Let's copy the `move_usafabot.sh` to the `my_scripts` folder you created earlier:

`cp move_usafabot.sh ~/master_ws/src/usafabot/usafabot_curriculum/my_scripts`

>**Syntax:** `cp <source> <destination>`

**Note:** for the above to work, you must already be in the same folder as the `move_usafabot.sh` file. Otherwise you have to use the complete file path, such as `~/my_folder/move_usafabot.sh`.

You can now delete your `my_folder` folder.

`cd ..`

`rm -r my_folder`

Change directories to your `my_scripts` folder. We can use a ROS tool, `roscd`, to change directories to ROS packages without using the complete file path:

`roscd usafabot_curriculum/my_scripts`

> **Syntax:** `roscd <package/folder>`

Edit the **move_usafabot_square.py** file and paste the following contents:

```python
#!/usr/bin/env python3
import rospy, time, math
from geometry_msgs.msg import Twist

class MoveUSAFABot():
    def __init__(self):
        self.pub = rospy.Publisher('cmd_vel', Twist, queue_size=1)
        self.cmd = Twist()
        self.ctrl_c = False
        rospy.on_shutdown(self.shutdownhook)
        self.rate = rospy.Rate(10)    # 10 Hz
        
    def publish_cmd_vel_once(self):
        """
        In case publishing fails on first attempt
        """
        while not self.ctrl_c:
            connections = self.pub.get_num_connections()
            if connections > 0:
                self.pub.publish(self.cmd)
                rospy.loginfo("Cmd Published")
                break
            else:
                self.rate.sleep()
                
    def shutdownhook(self):
        rospy.loginfo("Shutting down. Stopping USAFABot!")
        self.stop_usafabot()
        self.ctrl_c = True
        
    def stop_usafabot(self):
        self.cmd.linear.x = 0.0
        self.cmd.angular.z = 0.0
        self.publish_cmd_vel_once()
        
    def move_time(self, moving_time = 10.0, lin_spd = 0.2, ang_spd = 0.2):
        self.cmd.linear.x = lin_spd
        self.cmd.angular.z = ang_spd
        
        self.publish_cmd_vel_once()
        time.sleep(moving_time)
        
    def move_square(self):
        i = 0
        
        while not self.ctrl_c:
            # Move Forward
            self.move_time(moving_time = 2.0, lin_spd = 0.2, ang_spd = 0)
            # Turn
            ang_spd = 0.5    # rad/sec
            moving_time = math.radians(90)/ang_spd
            self.move_time(moving_time = moving_time, lin_spd = 0.0, ang_spd = ang_spd)
            
        
if __name__ == '__main__':
    rospy.init_node('move_usafabot')
    moveusafabot_object = MoveUSAFABot()
    try:
        moveusafabot_object.move_square()
    except rospy.ROSInterruptException:
        pass
```

The script is already executable, so you can run it using ROS!

`rosrun usafabot_curriculum move_usafabot_square.py`

**Note:** It won't be a perfect square as the robot doesn't turn perfectly, but it will be close!

The robot is now driving in a square until the script is killed. If you select the terminal and hit `ctrl+c`, it will kill the script.

Run the script again and this time hit `ctrl+z`. You can see that the robot is still running, but the commands are not updating. This is because `ctrl+z` suspends the current process, but does not kill it. We can observe all running processes on Linux by typing `ps -faux`. As you can see, there are a lot! The `grep` command allows us to filter these processes. Try the following:

`ps -faux | grep move_usafabot_square.py`

The first entry should be our process and the leftmost number is the process ID (PID). We can kill any process using this number and the `kill` command:

`kill PID` replacing PID with the number listed. If you hit enter again, you should see that the process was killed. Unfortunately, the USAFABot will just continue to execute the last command sent, so you need to kill the simulation as well. Just select that terminal and hit `ctrl+c`.

The `grep` tool is very powerful and can be used with any Linux command. If we wanted to see all usafabot packages available to us, we could type the following:

`rospack list`

There are a lot, so this isn't very helpful, but we can filter this command!

`rospack list | grep usafabot`

The vertical line, '|', pipes the results of the first command into the second command, so we can filter all packages looking only for usafabot packages.

### Working with a remote machine


### Summary
You have seen a lot of different Linux/ROS commands during this lesson and it only scratches the surface.